In [ ]:
import torch
import wandb
import numpy as np
from tqdm import trange
from IPython.display import clear_output


from src.models.base_texture_nca import BaseTextureNCA
from src.loss.vgg_loss import VGGLoss
from src.loss.mse_loss import MSELoss
from src.trainer.nca_trainer import NCATrainer
from src.utils.utils import *
from src.utils.EarlyStopping import EarlyStopping
from src.utils.OmniConfig import OmniConfig

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

wandb.login()

wandb: [wandb.login()] Loaded credentials for https://api.wandb.ai from C:\Users\GAI\_netrc.
wandb: Currently logged in as: apc582nntscott (apc582nntscott-nycu-gia-ail) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


True

# config

In [2]:
default_config = {
    'project_name': 'base_texture_nca',
    'run_name': 'sweep_run',
    'target_img_path': r'C:\Users\GAI\Desktop\Scott\NCA_Research\dataset\dotted_0201.jpg', # 請確認路徑存在
    'use_gradient_checkpoint': False, # 建議設為 True 以防 OOM
    # 以下為預設值，會被 Sweep 參數覆蓋
    'lr': 2e-3,
    'batch_size': 8,
    'hidden_n': 96,
    'chn': 12,
    'img_size': 128,
    'step_min': 32,
    'step_max': 96,
    'pool_reset_freq': 8,
    'loss_type': 'vgg',
    'seed': 42,
    'pool_size': 1024,
    'max_steps': 3000,
    'log_interval': 200,
    'early_stop_patience': 500,
    'early_stop_min_delta': 1e-4,
    'test_steps': 200,
    'test_patience': 50
}

In [3]:
sweep_configuration = {
    'method': 'bayes',
    'metric': {
        'name': 'test/final_loss',
        'goal': 'minimize'
    },
    'parameters': {
        'lr': {'min': 0.0005, 'max': 0.005, 'distribution': 'uniform'},
        'batch_size': {'values': [4, 8,]},
        'hidden_n': {'values': [24, 32, 64, 96, 128]},
        'step_min': {'values': [20, 32, 48, 64]},
        'pool_reset_freq': {'values': [4, 8, 12, 16]},
        
        # 強烈建議這裡先鎖定 'vgg'，如果要測 mse 請另外開一個 sweep
        'loss_type': {'values': ['vgg']}, 
        
        'seed': {'values': [42, 1024]},
        'img_size': {'values': [64, 96, 128]},
        'chn': {'values': [8, 12, 16]},
        'max_steps': {'value': 3000}
    }
}

# define logic

## setup case

In [4]:
def setup_case(config):
    """初始化所有訓練所需物件"""
    # 1. 設定種子
    set_seed(config.seed)
    
    # 2. 準備目標圖片 & Loss
    # 假設你有一個 target.jpg，若無可先用雜訊代替測試
    try:
        target_img = load_target_image(config.target_img_path, config.img_size, device)
    except FileNotFoundError:
        print(f"Warning: {config.target_img_path} not found. Using random target.")
        target_img = torch.rand(1, 3, config.img_size, config.img_size).to(device)
    
    # 3. 依照 config 選擇 Loss 函數
    if config.loss_type == 'mse':
        print("Using Simple MSE Loss")
        loss_fn = MSELoss(target_img)
    else:
        print("Using VGG / OT Loss")
        loss_fn = VGGLoss(target_img, device=device)

    # 4. 建立模型
    model = BaseTextureNCA(config).to(device)
    
    # 5. 建立優化器與排程器
    optimizer = torch.optim.Adam(model.parameters(), lr=config.lr)
    # 範例 Scheduler: 2000 步後降低學習率
    scheduler = torch.optim.lr_scheduler.MultiStepLR(optimizer, milestones=[2000], gamma=0.3)
    
    # 6. 建立資料池 (Data Pool)
    # 直接使用 Model 的 seed 方法產生初始池
    pool = model.seed(config.pool_size, sz=config.img_size)
    
    return model, optimizer, scheduler, pool, loss_fn

## train

In [5]:
def train(model, optimizer, scheduler, pool, loss_fn, config):
    """
    訓練迴圈 (Refactored)。
    包含 Config 驅動的步數、早停機制與模組化 Logging。
    """
    
    # 1. 初始化 Trainer
    trainer = NCATrainer(
        model=model,
        pool=pool,
        optimizer=optimizer,
        scheduler=scheduler,
        loss_fn=loss_fn,
        config=config,
        device=device
    )

    # 2. 從 Config 讀取訓練參數 (設定預設值以防漏填)
    max_steps = config.get('max_steps', 5000)
    log_interval = config.get('log_interval', 100)
    
    # 3. 初始化早停機制
    # NCA 訓練通常會有波動，建議 patience 設大一點 (例如 500-1000)
    early_stopper = EarlyStopping(
        patience=config.get('early_stop_patience', 1000),
        min_delta=config.get('early_stop_min_delta', 1e-5)
    )

    # 4. 開始訓練迴圈
    # 使用 trange 顯示進度條
    progress_bar = trange(max_steps, desc="Training")
    
    for i in progress_bar:
        # --- 訓練一步 ---
        stats = trainer.train_step(current_step=i)
        train_loss = stats['loss']
        
        # --- 準備 Log 資訊 ---
        log_dict = {
            "train/loss": train_loss,
            "train/lr": stats['lr'],
            "step": i
        }

        # --- 視覺化 (使用 Utils) ---
        if i % log_interval == 0:
            log_dict["train/image"] = log_batch_image(stats['batch_x'], step=i)

        # --- Wandb Log ---
        wandb.log(log_dict)
        
        # --- 更新進度條文字 ---
        if i % 10 == 0:
            progress_bar.set_description(f"Loss: {train_loss:.5f}")

        # --- 早停檢查 ---
        # 這裡示範基於 Train Loss 的早停 (若 Loss 長期卡住不動則停)
        if early_stopper(train_loss):
            print(f"\n[Early Stopping] Triggered at step {i}. Best Loss: {early_stopper.best_loss:.5f}")
            break

## test

In [6]:
def test(model, loss_fn, config):
    """
    測試/生成階段：產生演化影片，並記錄 Loss 曲線與執行早停。
    """
    print("Testing (Generating Video & Monitoring Loss)...")
    
    # 1. 準備設定
    test_steps = config.get('test_steps', 300)      # 測試總步數
    patience = config.get('test_patience', 50)      # 測試階段的早停耐心值
    img_size = config.get('img_size', 128)
    
    # 2. 初始化早停器 (這裡設定 min_delta 稍小，目的是檢測收斂)
    stopper = EarlyStopping(patience=patience, min_delta=1e-5)
    
    model.eval() # 設定為評估模式
    
    with torch.no_grad():
        # 初始化種子
        x = model.seed(1, sz=img_size)
        
        frames = []
        test_losses = []
        
        iterator = trange(test_steps, desc="Generating")
        
        for i in iterator:
            # A. 模型推論
            x = model(x)
            
            # B. 計算 Loss (用於評估品質與穩定性)
            # 注意：NCA 輸出 x 是 [1, 12, H, W]，Loss 需要 RGB [1, 3, H, W]
            rgb_x = to_rgb(x)

            # 計算 Loss (MSE 或 VGG)
            # 因為 x 是 batch=1，直接取 item()
            current_loss = loss_fn(rgb_x).item()
            test_losses.append(current_loss)
            
            # C. 處理圖片 (存入 Frames)
            # [1, 3, H, W] -> [H, W, 3] -> Numpy
            img = rgb_x[0].permute(1, 2, 0).cpu().numpy()
            img = np.clip(img, 0, 1)
            frames.append((img * 255).astype(np.uint8))
            
            # D. 更新進度條資訊
            iterator.set_postfix(loss=current_loss)
            
            # E. 安全檢查：如果 Loss 變成 NaN 或無限大，立刻停止 (NCA 常見問題)
            if np.isnan(current_loss) or np.isinf(current_loss):
                print(f"⚠️ Test stopped early due to instability (NaN/Inf) at step {i}")
                break

            # F. 早停機制：如果圖片已經穩定不再變化 (Loss 收斂)
            if stopper(current_loss):
                print(f"✅ Test stopped early (Converged) at step {i}. Best Loss: {stopper.best_loss:.5f}")
                break
                
        # 3. 記錄到 WandB
        # 轉換影片格式 [T, H, W, C] -> [T, C, H, W]
        frames = np.array(frames).transpose(0, 3, 1, 2)
        
        # 建立 Log Dictionary
        log_dict = {
            "test/video": wandb.Video(frames, fps=30, format="mp4"),
            "test/final_loss": test_losses[-1] if test_losses else 0,
            "test/steps_taken": len(frames)
        }
        
        # 繪製 Loss 曲線 (可以看穩定性)
        data = [[s, l] for s, l in enumerate(test_losses)]
        table = wandb.Table(data=data, columns=["step", "loss"])
        log_dict["test/loss_curve"] = wandb.plot.line(
            table, "step", "loss", title="Test Generation Stability"
        )
        
        wandb.log(log_dict)
        
    model.train() # 恢復訓練模式 (如果是 pipeline 中途測試的話)

## pipeline

In [7]:
def model_pipeline(hyperparams):
    """主流程"""
    # 這裡傳入 config=hyperparams 讓 wandb 幫我們管理 config
    full_run_name = get_next_run_name(
        project_name=hyperparams['project_name'], 
        base_name=hyperparams.get('run_name', None)
    )
    with wandb.init(project=hyperparams['project_name'], config=hyperparams, name=full_run_name):
        # 使用 wandb.config (這樣如果用 Sweeps 調整參數會自動生效)
        config = wandb.config
        
        # 1. Setup
        model, optim, sched, pool, loss_fn = setup_case(config)
        
        # 2. Train
        train(model, optim, sched, pool, loss_fn, config)
        
        # 3. Test
        test(model,loss_fn, config)
        
    return model

## sweep

In [8]:
def train_sweep(config=None):
    # 1. 初始化 WandB，這裡不傳入 config，讓 wandb 自動從 sweep controller 抓取
    with wandb.init(config=config):
        # 取得 sweep 這次派發的參數
        sweep_config = wandb.config
        
        # 2. [關鍵步驟] 合併 Config
        # 我們複製一份預設 config，然後用 sweep 的參數覆蓋它
        # 這樣沒被 sweep 到的參數 (如路徑、pool_reset_freq) 依然存在
        merged_config = default_config.copy()
        merged_config.update(sweep_config)
        merged_config = OmniConfig(merged_config)
        
        # 設定 Run Name (可選：加上參數資訊方便識別)
        run_name = f"Sweep_H{merged_config['hidden_n']}_lr{merged_config['lr']:.4f}"
        wandb.run.name = run_name
        
        print(f"🚀 Starting Sweep Run: {run_name}")
        print(f"Hyperparams: {dict(sweep_config)}")

        # 3. 執行標準流程 (完全復用你原本的函式)
        try:
            # Setup
            model, optim, sched, pool, loss_fn = setup_case(merged_config)
            
            # Train
            train(model, optim, sched, pool, loss_fn, merged_config)
            
            # Test
            test(model, loss_fn, merged_config)
            
            # 例如在訓練迴圈結束後
            print("訓練結束！")
            print(f"Wandb 紀錄的最終 Training Loss: {wandb.run.summary.get('train/loss')}")
            print(f"Wandb 紀錄的最終 Test Loss: {wandb.run.summary.get('test/final_loss')}")
            clear_output(wait=True)
        except Exception as e:
            # 如果參數太激進導致爆顯存或報錯，捕捉異常避免整個 Sweep 停掉
            print(f"❌ Run Failed: {e}")
            pass

# run case

In [ ]:
# A. 註冊 Sweep ID (只需執行一次)
# 這會在 WandB 伺服器建立一個 Sweep Controller
sweep_id = wandb.sweep(sweep_configuration, project=default_config['project_name'])

print(f"Sweep ID: {sweep_id}")

# B. 啟動 Agent (開始跑實驗)
# count=10 表示跑 10 次實驗就停止
wandb.agent(sweep_id, function=train_sweep, count=50)

Create sweep with ID: gg59kyu9
Sweep URL: https://wandb.ai/apc582nntscott-nycu-gia-ail/base_texture_nca/sweeps/gg59kyu9
Sweep ID: gg59kyu9


wandb: Agent Starting Run: ic2hle2q with config:
wandb: 	batch_size: 4
wandb: 	chn: 8
wandb: 	hidden_n: 64
wandb: 	img_size: 64
wandb: 	loss_type: vgg
wandb: 	lr: 0.002196126079477388
wandb: 	max_steps: 3000
wandb: 	pool_reset_freq: 16
wandb: 	seed: 1024
wandb: 	step_min: 20
wandb: [wandb.login()] Loaded credentials for https://api.wandb.ai from C:\Users\GAI\_netrc.


🚀 Starting Sweep Run: Sweep_H64_lr0.0022
Hyperparams: {'batch_size': 4, 'chn': 8, 'hidden_n': 64, 'img_size': 64, 'loss_type': 'vgg', 'lr': 0.002196126079477388, 'max_steps': 3000, 'pool_reset_freq': 16, 'seed': 1024, 'step_min': 20}
Using VGG / OT Loss


Loss: 82495.66406: 100%|██████████| 3000/3000 [05:38<00:00,  8.86it/s]        


Testing (Generating Video & Monitoring Loss)...


...

訓練結束！
Wandb 紀錄的最終 Training Loss: 104518.90625
Wandb 紀錄的最終 Test Loss: 1898530.875
❌ Run Failed: name 'clear_output' is not defined


step,▁▁▂▂▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇▇▇▇█
test/final_loss,▁
test/steps_taken,▁
train/loss,█▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
train/lr,█████████████████████████████▁▁▁▁▁▁▁▁▁▁▁
step,2999
test/final_loss,1898530.875
test/steps_taken,105
train/loss,104518.90625
train/lr,0.00061


wandb: Agent Starting Run: b4kcbp28 with config:
wandb: 	batch_size: 4
wandb: 	chn: 16
wandb: 	hidden_n: 64
wandb: 	img_size: 96
wandb: 	loss_type: vgg
wandb: 	lr: 0.0008038318154109087
wandb: 	max_steps: 3000
wandb: 	pool_reset_freq: 12
wandb: 	seed: 1024
wandb: 	step_min: 20
wandb: [wandb.login()] Loaded credentials for https://api.wandb.ai from C:\Users\GAI\_netrc.


🚀 Starting Sweep Run: Sweep_H64_lr0.0008
Hyperparams: {'batch_size': 4, 'chn': 16, 'hidden_n': 64, 'img_size': 96, 'loss_type': 'vgg', 'lr': 0.0008038318154109087, 'max_steps': 3000, 'pool_reset_freq': 12, 'seed': 1024, 'step_min': 20}
Using VGG / OT Loss


Loss: 113794.82812:  95%|█████████▍| 2836/3000 [05:34<00:17,  9.57it/s]  